# Finding The Best Markets To Advertise In - Visualization

This notebook is part of a bigger project called "Finding the Best Markets to Advertise In". Due to limitations while uploading the file to GitHub I was forced to split the notebooks in two. This might by an inconvenience, but it also allows for lighter, more compact notebooks. 

In the notebook "Finding The Best Markets To Advertise In - Analysis", we found out what were the best countries to advertise in, that is, the countries with the highest number of coders as well as checked how much every coder spends on average in learning.

However, these informations weren't enough to decide based solely on data, what are the best markets. For this reason, it would be necessary to send this analysis to a sales or marketing specialist to make a decision based on his expertise. However, we can help his decision making by offering his the grouds to make a choice.

This "help" will be an interactive map that summarises the information we have so far.

# Creating an Interactive Map



In [1]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import folium
import altair as alt

## Importing the Top Countries DataFrame

The top countries dataframe originates from the "Finding the Best Markets to Advertise In - Analysis" notebook. It contains information on the number of survey takers by country.

In [2]:
top_countries = pd.read_csv(r"C:\Users\celio\Data Analysis\Projects\FInding the Best Markets to Advertise In\top_countries.csv")
top_countries.head()

,Unnamed: 0,CountryLive,n_survey_takers
0,0,United States of America,5791
1,1,India,1400
2,2,United Kingdom,757
3,3,Canada,616
4,4,Brazil,364


## Importing the Top Markets Data Frame

This DataFrame also originates from the "Finding the Best Markets to Advertise In - Analysis". It's a clean version of original dataframe containing only the survey results concerning the top countries,plus the column "MonthBudget", which represents the average monthly amount spent by each survey taker on learning. 

In [3]:
top_markets = pd.read_csv(r"C:\Users\celio\Data Analysis\Projects\FInding the Best Markets to Advertise In\top_markets.csv")
top_markets.head()

,Unnamed: 0,level_0,level_1,Age,AttendedBootcamp,BootcampFinish,BootcampLoanYesNo,BootcampName,BootcampRecommend,ChildrenNumber,...,YouTubeFunFunFunction,YouTubeGoogleDev,YouTubeLearnCode,YouTubeLevelUpTuts,YouTubeMIT,YouTubeMozillaHacks,YouTubeOther,YouTubeSimplilearn,YouTubeTheNewBoston,MonthBudget
0,0,United States of America,1,34.0,0.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13.333333
1,1,United States of America,2,21.0,0.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,1.0,1.0,NaN,NaN,NaN,NaN,NaN,200.000000
2,2,United States of America,7,29.0,0.0,NaN,NaN,NaN,NaN,NaN,...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.000000
3,3,United States of America,8,23.0,0.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,24.137931
4,4,United States of America,10,20.0,0.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,0.000000


## Importing The Coordinates

Technically, we are not importing the coordinates, but a json file containing the geographic data of the countries. This file can be layed over the map to create a choropleth.

In [4]:
# We're actually not importing coordinates, but a geojson file to lay over the map
# With the Geojson, we can build a choropleth file.
layer = gpd.read_file(r"C:\Users\celio\Data Analysis\Projects\FInding the Best Markets to Advertise In\custom.geo.json")
layer =layer[["admin","name","geometry"]] # We're slicing the Geopandas DF because that's everything we need.
layer.head(3)

,admin,name,geometry
0,Albania,Albania,"POLYGON ((20.59025 41.85540, 20.46318 41.51509..."
1,Austria,Austria,"POLYGON ((16.97967 48.12350, 16.90375 47.71487..."
2,Belgium,Belgium,"POLYGON ((3.31497 51.34578, 4.04707 51.26726, ..."


# Code Events, Podcasts, Resources and Youtube

The top_markets DataFrame offers information on what code events survey takers attended, as well as podcasts and youtube channels they listen/watch. Given the high number of events, youtube channels and podcasts there are, it's hard to make a decision based only on this data, however this information may help the marketing team when looking for events, podcasts or youtube channels to sponsor as advertising. 

We'll retrieve all this information from the dataset and then we'll plot it in a map to facilitate navigation.

### Function To Retrieve The Data

The function below takes a string as an argument and slices the top_market data frame we have been working with. It than transforms it into a pivot_table to find out how many survey takers attended each event. After that, the function finds the first 5 most attended events or resources or podcasts or youtube channels, depending on the string value passed, and plots a chart.

In [5]:
def find_object(string_value): # Takes string values
    
    cols = ["CountryLive"] # Instiation of a list with one object (CountryLive) - will be used to slice DF
    
    for c in top_markets.columns: # Loops through the columns of the top_markets DataFrame
        if string_value in c: # Checks if the string value is in the DataFrame
            cols.append(c) # if True, appends the value to cols list.
            
    df = top_markets[cols] # slices the DataFrame with the cols list
    df_pivot = df.pivot_table(index="CountryLive",aggfunc="sum") # creates a pivot table 
                                            # Groups by country and sums up number of survey takers who attended each event
    
    charts = dict.fromkeys(df_pivot.index) # Instantiates a dictionary to store chart
    for obj in df_pivot.index: # loops through the countries in the df_pivot_index
        most_popular = df_pivot.loc[obj].sort_values(ascending =False).head(5).reset_index() # selects top 5 results
        most_popular.columns = [string_value,"n_survey_takers"] # changes column names
        # Plot Chart
        charts[obj] = alt.Chart(most_popular,
                               title = "Most Frequented {} in {}".format(string_value,obj)).mark_bar().encode( 
                                # Title uses string value and country name for formating
                                
                                y = alt.Y(string_value,axis = alt.Axis(labelFontSize=12)),
                                
                                x= alt.X("n_survey_takers",axis = alt.Axis(title = "Number of Participants")),
                                
                                color = string_value)
        
    return charts # returns a dictionary

### Events

Below there is an extensive example of how it works with the "CodeEvent" string argument.

In [6]:
events= find_object("CodeEvent")
alt.vconcat(events["Australia"],events["Brazil"],events["Canada"],
           events["France"],events["Germany"],events["India"],events["Poland"],
           events["Russia"],events["United Kingdom"],events["United States of America"])

alt.VConcatChart(...)

### Podcasts

Small example with the "Podcast" argument.

In [7]:
podcasts = find_object("Podcast")
podcasts["Brazil"]


alt.Chart(...)

### Resources

In [8]:
resources = find_object("Resource")
resources["United States of America"]

alt.Chart(...)

### Youtube

In [9]:
youtube = find_object("YouTube")
youtube["Brazil"]

alt.Chart(...)

At the present moment, we have all the data about the events, podcasts, youtube channels and resources stored in their respectives dictionaries. This will be useful when plotting the map. But before doing this, we have to merge the dataframe with the geographic information and the data we want to display on the map.

## Cleaning Geographic Data

To plot the map, we have to make sure that the countries in the Geopandas DataFrame and the top_countries DataFrame are written the same way. 

The first step to this, is checking for "duplicates".

In [10]:
# There are no perfect duplicates on the DataFrame, however some countries may have different names or spelling
# Like Serbia and Republic for Serbia. So we'll check for those

for country in top_countries["CountryLive"].values:
    # Parameter is a selection of all entries containing the name of a country
    parameter = top_countries.loc[top_countries["CountryLive"].str.contains(country),"CountryLive"]
    if parameter.count() > 1: #If the number of matches is higher than 1, it's probably a problematic name
        print(parameter,"\n")

1                         India
151    British Indian Ocean Ter
Name: CountryLive, dtype: object 

21                Serbia
70    Republic of Serbia
Name: CountryLive, dtype: object 

11     Nigeria
104      Niger
Name: CountryLive, dtype: object

C:\Anaconda\envs\data\lib\site-packages\pandas\core\strings.py:1954: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


 

57     Dominican Republic
125              Dominica
Name: CountryLive, dtype: object 



Apparently, the only "duplicated" country is Serbia, which has entries as Serbia and Republic of Serbia. We'll sum the result of both countries and keep only the first entry.

The other entries refer only to countries with similar names. 

In [11]:
top_countries[top_countries["CountryLive"].str.contains("Serbia")]

,Unnamed: 0,CountryLive,n_survey_takers
21,21,Serbia,115
70,70,Republic of Serbia,24


In [12]:
top_countries.loc[top_countries["CountryLive"]=="Serbia","n_survey_takers"]= 115+24

In [13]:
top_countries = top_countries.drop(70)

Now we can get back to the main test, which is to find out what countries are featured in the top_countries DataFrame that aren't in the layer DataFrame.

In [14]:
test_df=pd.merge(left=top_countries,right=layer,
                left_on="CountryLive",right_on="name",
                how="left")

missing = test_df[test_df["name"].isnull()]
missing

,Unnamed: 0,CountryLive,n_survey_takers,admin,name,geometry
0,0,United States of America,5791,NaN,NaN,None
16,16,"Netherlands (Holland, Europe)",142,NaN,NaN,None
31,31,Singapore,75,NaN,NaN,None
34,34,Czech Republic,72,NaN,NaN,None
47,47,Great Britain,45,NaN,NaN,None
53,53,Bosnia & Herzegovina,40,NaN,NaN,None
57,57,Dominican Republic,32,NaN,NaN,None
64,64,Korea South,29,NaN,NaN,None
66,66,Hong Kong,25,NaN,NaN,None
80,81,Virgin Islands (USA),15,NaN,NaN,None


Now that we know what are the problematic entries, we can modify the name of the countries accordingly. To make the work easier, we'll only work with the entries containing at least 20 survey takers.

### Fixing Missing Data

Some countries in the top_countries dataframe do not match any entries in the layer DataFrame (which contains the geographic information). We have to investigate this further, in order to bind both datasets together.

In [15]:
# Creating a new DataFrame with the names of the countries
problematic = missing.loc[missing["n_survey_takers"]>=20,"CountryLive"].str.split(expand=True)

for name in problematic[0]:
    root_of_problem = layer.loc[layer["name"].str.contains(name),"name"]
    print(problematic.loc[problematic[0]==name],"\n",
          root_of_problem,"\n\n")

        0       1   2        3
0  United  States  of  America 
 13           United Kingdom
93     United Arab Emirates
162           United States
Name: name, dtype: object 


              0          1        2     3
16  Netherlands  (Holland,  Europe)  None 
 27    Netherlands
Name: name, dtype: object 


            0     1     2     3
31  Singapore  None  None  None 
 Series([], Name: name, dtype: object) 


        0         1     2     3
34  Czech  Republic  None  None 
 7    Czech Rep.
Name: name, dtype: object 


        0        1     2     3
47  Great  Britain  None  None 
 Series([], Name: name, dtype: object) 


         0  1            2     3
53  Bosnia  &  Herzegovina  None 
 4    Bosnia and Herz.
Name: name, dtype: object 


            0         1     2     3
57  Dominican  Republic  None  None 
 153    Dominican Rep.
Name: name, dtype: object 


        0      1     2     3
64  Korea  South  None  None 
 113              Korea
124    Dem. Rep. Korea
Name: name, dtype

The result might not be very clear, but basically:

1) United States of America in the top_countries is written as United States in the name column of the layer DF

2) Netherlands (Holland,Europe) equals Netherlands

3) No entries were found for Great Britain, Singapore und Hong Kong

4) Czech Republic is Czech Rep.

5) Dominican Republic is Dominican Rep.

6) South Korea has become Korea

We'll correct the entries for finally merge the DataFrames and plot the map.

In [16]:
layer.loc[layer["name"]=="United States","name"] = "United States of America"
top_countries.loc[top_countries["CountryLive"]=="Netherlands (Holland, Europe)","CountryLive"] = "Netherlands"
layer.loc[layer["name"]=="Czech Rep.","name"] = "Czech Republic"
top_countries.loc[top_countries["CountryLive"]=="Bosnia & Herzegovina","CountryLive"] = "Bosnia and Herz."
layer.loc[layer["name"]=="Dominican Rep.","name"]= "Dominican Republic"
layer.loc[layer["name"]=="Korea","name"] = "Korea South"

We can check the results:

In [17]:
check = pd.merge(left=top_countries,right=layer,
                left_on="CountryLive",right_on="name",
                how="left")
check[check["name"].isnull()]

,Unnamed: 0,CountryLive,n_survey_takers,admin,name,geometry
31,31,Singapore,75,NaN,NaN,None
47,47,Great Britain,45,NaN,NaN,None
66,66,Hong Kong,25,NaN,NaN,None
80,81,Virgin Islands (USA),15,NaN,NaN,None
98,99,Malta,6,NaN,NaN,None
104,105,Mauritius,5,NaN,NaN,None
105,106,Republic of Montenegro,5,NaN,NaN,None
107,108,Trinidad & Tobago,5,NaN,NaN,None
108,109,Cote D'Ivoire,5,NaN,NaN,None
116,117,Maldives,4,NaN,NaN,None


It worked! We're helpless concerning Singapore and Hong Kong. If these countries are not featured in the map with the geojson layer, there is little we can do about them. The other countries on the list all have less than 20 survey takers.

# Merging DataFrames

Now that we cleaned the data, we are able to plot the map. The final step is merging the clean versions of top_countries and layer.

In [18]:
geo = pd.merge(left=layer,right=top_countries,
              left_on="name",right_on="CountryLive",
              how="left") 

# """Notice that we inverted the order of the DataFrames. Layer is now on the left side of merging
# while top_countries is on the right side. Merging like this allows the information to be stored in a geopandas DF
# instead of a regular df, which makes easier for folium (the library that plots the map) to read it"""

# This was done manually but for more entries, we could have used the googlemaps API
# On more about the usage of Googlemaps API, I recommend taking a look at my Notebook Open Legal Data

coordinates = {"Australia":[-33.8469759,150.37],
              "Brazil":[-23.6815314,-46.8],
              "Canada": [43.7184038,-79.5],
              "France":[48.8589507,2.27],
              "Germany":[52.5069312,13.14],
              "India":[19.0825223,72.741],
               "Poland":[52.2330269,20.78],
               "Russia":[55.5815244,36.82],
              "United Kingdom":[51.5287352,-0.3817],
              "United States of America":[38.89378,-77.15]} ## Instanting a Dictionary With Coordinates of Capital Cities

layers = {"Events":events,"Podcasts":podcasts,"Resources":resources,"Youtube":youtube}

In [19]:
earth = folium.Map(location=[49,13],zoom_start=2) # This instatiates the Folium map
# It's important that the map is instatiated before the function below is run, otherwise it won't work.

In [20]:
#FUNCTION TO CREATE MARKERS THAT WILL BE ADDED TO THE MAP
def create_markers(keyword,color):
    
    fg = folium.FeatureGroup(name = keyword).add_to(earth)
    d = layers.get(keyword)
    for key in d:
        marker = folium.Marker(
            location = coordinates[key],
            icon = folium.Icon(color = color),
            tooltip = key,
            popup = folium.Popup(max_width=500).add_child(
                    folium.VegaLite(data =podcasts[key],
                                   width =450,
                                   height = "80%"))
                          ).add_to(fg)

# Interactive Map With Target Groups

The map below summarizes the following information:

## 1) The Number of Survey Takers By Country
    
    - This is shown in the choropleth map. The darker the color of a country, the higher the number of survey takers located there.
    
    - To visualize the number of survey takers, it suffices to hoover the mouse over the country.
    
## 2) The main communication channels for the top countries.

    - Every channel (Event, Podcast, Resource, Youtube) is accessible through the layer control at the right of the map
    
    - The top 5 most attended/heard/consumed/watched events/podcasts/resources/youtube channels are show upon clicking the marker of each country.

In [21]:
choropleth = folium.Choropleth(geo_data = geo,
                              data = geo,
                              columns = ["CountryLive","n_survey_takers"],
                              key_on="feature.properties.name", # This is tricky and it's worst part of using folium
                                                                # Regardless of how the json file is organised
                                                               # this string always starts with feature (without an s)
                                                               # even if the json file says features (with an s)
                                                               # the rest of the string is JavaScript Notation
                                                               # according to the JSON file
                                                               # Best way to find out the string is play with the
                                                               # json file via the python json library
                               
                               bins = 9, # Unfortunately Folium Only Support 9 Bins, more would have been better
                               fill_color = "YlGn",
                              highlight=True,
                              legend_name = "Number Of Survey Takers by Country",
                              name = "Choropleth").add_to(earth)

event_markers = create_markers("Events","cadetblue")
podcast_markers = create_markers("Podcasts","orange")
resource_markers = create_markers("Resources","green")
youtube_markers = create_markers("Youtube","red")


choropleth.geojson.add_child(
    folium.features.GeoJsonTooltip(["CountryLive","n_survey_takers"],
                                   aliases = ["Country", "Number of Participants"]))


folium.LayerControl().add_to(earth)

earth

# Final Conclusions

In the first part of th project, we tried to find out the **best countries to advertise in**. We analyzed the results of a survey conducted by freeCodeCamp. After verifying whether the dataset contained relevant data for our goal or not by **checking if the job interests of survey takers match the products sold by the company** (an e-learning company specialized in web-design, game developing, programming, data science and etc), we found out the countries with the highest density of survey takers. 

The countries with the highest number of survey takers are: **"United States, India, United Kingdom, Canada, Brazil, Germany, Poland, Russia, Australia and France".**

Then we checked in what countries survey takers spend the most on learning. For this analysis, we considered only the countries mentioned above. Specially interesting were the results in **Australia, Poland and Russia**, where, in spite of the small number of survey takers, the average amount spent by survey takers was very high.

At this point, it was clear that **one of the markets for advertisement should be the United States**, because of the high number both of survey takers and amount spent for learning. Nevertheless, we'd **recommend sending the results of this analysis to the marketing specialist** so they can participate on the decision making, since there are many variables to consider, for example, the **language spoken** in each country or the GDP and **other measures of social development**.

Finally, to help decision making and summarize information, we created an **interactive map** showing the number of survey takers in each country, as well as the most common events, podcasts, resources and youtube channels in the top countries.